In [ ]:
pip install boto3 pandas tensorflow

In [4]:
import boto3
import pandas as pd
from decimal import Decimal
from datetime import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [ ]:
# Initialize DynamoDB client
client = boto3.client('dynamodb')

# Fetch all data from DynamoDB table
table_name = 'PowerConsumptionTable'

# Scan table to get all items
response = client.scan(TableName=table_name)
data = response['Items']

# Convert to DataFrame
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'].apply(lambda x: x['S']))
df['entry_no'] = df['entry_no'].apply(lambda x: int(x['N']))
df['Power_Consumption'] = df['Power_Consumption'].apply(lambda x: float(x['N']))

# Sort by timestamp
df.sort_values(by='timestamp', inplace=True)
df.set_index('timestamp', inplace=True)

print(df)


In [ ]:
# Select features
features = df[['Power_Consumption']].values

# Normalize the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 4  # Number of time steps
X, y = create_sequences(scaled_features, seq_length)

# Split data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=1, validation_split=0.2)


plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Inverse transform to get actual values
y_test_actual = scaler.inverse_transform(y_test)
y_pred_actual = scaler.inverse_transform(y_pred)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(y_test_actual, label='Actual Power Consumption')
plt.plot(y_pred_actual, label='Predicted Power Consumption')
plt.xlabel('Time')
plt.ylabel('Power Consumption')
plt.legend()
plt.show()